In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
logs = []
for i in range(1,9):
    with open(f"Tablet/log_id_{i}.json") as f:
        log = json.load(f)
        for l in log:
            logs.append(l)

with open(f"Tablet/log_id_video.json") as f:
    log = json.load(f)
    for l in log:
        logs.append(l)
df = pd.DataFrame(logs)

In [3]:
df["environment"] = "tablet"
df.to_csv('Tablet/userTestEvents.csv', index=False)

In [4]:
# df["event"].count()
# df[df["event"] == "void_selection"]
# df["id"] = np.where(df["id"] == "10", 0, df["id"])
df["id"] = df["id"].where(df["id"] != "11", 0)

In [5]:
# logs = []
# for i in range(1,9):
#     with open(f"webfiles/reqs/RAreq{i}.json") as f:
#         log = json.load(f)
#         for l in log:
#             logs.append(l)
# dfr = pd.DataFrame(logs)

In [6]:
# dfr.to_csv('chartReqs.csv', index=False)

In [7]:
# endTime = df[(df["event"] == "start_evaluation") & (df["task"] == 1)]["timestamp"].values # no start enough
startTime = df[(df["task"] == 1)].groupby("id").first()["timestamp"].values
endTime = df[(df["task"] == 2)].groupby("id").first()["timestamp"].values

# finalTime = df[(df["event"] == "finish_task") & (df["task"] == 5)]["timestamp"].values
# totalTimes = (finalTime - startTime) / 1000
totalTimes = (endTime - startTime) / 1000
totalTimes

array([ 6.23 ,  5.496,  5.399, 40.531, 86.045, 21.72 , 22.856,  8.184,
       29.06 ])

In [8]:
# "ID_PARTICIPANTE,TASK_NUMBER,TASK_COMPLETION_TIME,NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK"
ids = df["id"].unique()

dfs = pd.DataFrame({})
for task in range(1,5):        
    startTime = df[(df["task"] == task)].groupby("id").first()["timestamp"].values
    endTime = df[(df["task"] == task+1)].groupby("id").first()["timestamp"].values
    taskTimesSeconds = (endTime - startTime) / 1000
    clicks = df[(df["task"] == task)].groupby("id")["event"].count().values
    dfs = pd.concat([dfs, pd.DataFrame({"ID_PARTICIPANTE": ids, "TASK_NUMBER": task, "TASK_COMPLETION_TIME": taskTimesSeconds,
                                       "NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK":clicks})])

startTime = df[(df["task"] == 5)].groupby("id").first()["timestamp"].values
endTime = df[(df["task"] == 5)].groupby("id").last()["timestamp"].values
taskTimesSeconds = (endTime - startTime) / 1000
clicks = df[(df["task"] == 5)].groupby("id")["event"].count().values
dfs = pd.concat([dfs, pd.DataFrame({"ID_PARTICIPANTE": ids, "TASK_NUMBER": 5, "TASK_COMPLETION_TIME": taskTimesSeconds,
                                   "NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK":clicks})])
dfs["environment"] = "tablet"
dfs

,ID_PARTICIPANTE,TASK_NUMBER,TASK_COMPLETION_TIME,NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK,environment
0,1,1,6.230,2,tablet
1,2,1,5.496,1,tablet
2,3,1,5.399,1,tablet
3,4,1,40.531,3,tablet
4,5,1,86.045,29,tablet
5,6,1,21.720,2,tablet
6,7,1,22.856,9,tablet
7,8,1,8.184,2,tablet
8,0,1,29.060,3,tablet
0,1,2,13.853,4,tablet


In [9]:
dfs.to_csv('Tablet/userTestMonitoringTablet.csv', index=False)

In [10]:
#df["event"].unique()

In [11]:
dfar = pd.read_csv("AR/userTestEvents.txt")
dfar["id"] = dfar["ID_PARTICIPANTE"]
dfar["task"] = dfar["TASK_NUMBER"]

dfar["TIME_OF_EVENT"] = dfar["TIME_OF_EVENT"].apply(lambda a: a[len("738579."):-len("7230")])
dfar["TIME_OF_EVENT"] = np.where((dfar["id"] == "1") | (dfar["id"] == "2"),
                                 dfar["TIME_OF_EVENT"].apply(lambda a: "2023-03-09 " + a),
                                 dfar["TIME_OF_EVENT"].apply(lambda a: "2023-03-10 " + a))
dfar["TIME_OF_EVENT"] = pd.to_datetime(dfar["TIME_OF_EVENT"])
dfar = dfar.rename(columns={"EVENT": "fullEvent", "TIME_OF_EVENT": "time"})
dfar

,ID_PARTICIPANTE,TASK_NUMBER,time,TASK_CLICK_NUMBER,fullEvent,id,task
0,0,1,2023-03-10 19:09:33.732,1,Open: 260,0,1
1,0,1,2023-03-10 19:09:35.606,2,PersonalInfo/GeneralStationInfo/Buttons/CycleT...,0,1
2,0,1,2023-03-10 19:09:37.340,3,PersonalInfo/CycleTimesMain/TitleBar/Buttons/B...,0,1
3,0,2,2023-03-10 19:09:40.271,1,HandMenu/LineMonitoringMenu/UsabilityTaskBar/B...,0,2
4,0,2,2023-03-10 19:09:42.119,2,PersonalInfo/GeneralStationInfo/Buttons/KPIsBu...,0,2
...,...,...,...,...,...,...,...
275,8,5,2023-03-10 16:14:56.452,4,PersonalInfo/GeneralStationInfo/Buttons/KPIsBu...,8,5
276,8,5,2023-03-10 16:14:58.075,5,PersonalInfo/KPIsMain/Buttons/PartCountButton,8,5
277,8,5,2023-03-10 16:15:08.476,6,PersonalInfo/KPIsSpecific/ShiftNumberSlider,8,5
278,8,5,2023-03-10 16:15:24.814,7,PersonalInfo/KPIsSpecific/TitleBar/Buttons/Clo...,8,5


In [12]:
with open(f"AR/eventsARList.json") as f:
    arevents = json.load(f)

In [13]:
# day = pd.to_datetime(df[df["id"] != 0]["timestamp"],unit='ms')
# day
# pd.concat([df,day],axis=1).groupby("id").first()

In [14]:
subkeys = ["event", "context", "subcontext", "attr"]
for key in list(arevents.keys())[1:]:
    are = arevents[key]
    evs = [are.get(sub, None) for sub in subkeys]
    dfar.loc[dfar["fullEvent"] == key, subkeys] = evs

for station in ["260", "270", "290"]:
    dfar.loc[dfar["fullEvent"] == f"Open: {station}", ["event", "station"]] = ["select_station", station] 
dfar["environment"] = "ar"

In [15]:
dfar.to_csv('AR/userTestEvents.csv', index=False)

In [16]:
dfmon = pd.read_csv("AR/userTestMonitoringAR.txt")
dfmon["TASK_COMPLETION_TIME"] = dfmon["TASK_COMPLETION_TIME"].apply(lambda a: float(a[len("00:00:"):]))
dfmon["environment"] = "ar"
dfmon.to_csv("AR/userTestMonitoringAR.csv", index=False)

In [17]:
dfar = dfar.drop(['ID_PARTICIPANTE', 'TASK_NUMBER'], axis=1)

In [18]:
finalMon = pd.concat([dfmon, dfs])
finalMon.to_csv("datasets/userTestMonitoring.csv", index=False)

df = df.rename(columns={"timestamp":"time"})
df["time"] = pd.to_datetime(df["time"], unit='ms')
df["id"] = df["id"].apply(lambda a: str(a))
dfar["id"] = dfar["id"].apply(lambda a: str(a))
finalTest = pd.concat([dfar, df])
finalTest
finalTest.to_csv('datasets/userTestEvents.csv', index=False)

In [19]:
df["id"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '0'], dtype=object)

In [20]:
finalTest["id"].unique()

array(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype=object)